## Load Datasets

In [4]:
import pandas as pd
import numpy as np

# Read in our data, and fill missing values
data = pd.read_csv("./data/clean_weather.csv", index_col=0)
data = data.ffill()

# Display a sequence of temperatures
data.head(10)

,tmax,tmin,rain,tmax_tomorrow
1970-01-01,60.0,35.0,0.00,52.0
1970-01-02,52.0,39.0,0.00,52.0
1970-01-03,52.0,35.0,0.00,53.0
1970-01-04,53.0,36.0,0.00,52.0
1970-01-05,52.0,35.0,0.00,50.0
1970-01-06,50.0,38.0,0.00,52.0
1970-01-07,52.0,43.0,0.00,56.0
1970-01-08,56.0,49.0,0.24,54.0
1970-01-09,54.0,50.0,0.40,57.0
1970-01-10,57.0,50.0,0.00,57.0


In [5]:
data["tmax"].head(10)

1970-01-01    60.0
1970-01-02    52.0
1970-01-03    52.0
1970-01-04    53.0
1970-01-05    52.0
1970-01-06    50.0
1970-01-07    52.0
1970-01-08    56.0
1970-01-09    54.0
1970-01-10    57.0
Name: tmax, dtype: float64

## RNN Architecture

![img](https://xiaophai-typora.oss-cn-shanghai.aliyuncs.com/2256672-cf18bb1f06e750a4.jpg)

## Full Forward Pass

### Initialize weight matrixes

$$
input\ layer: U_{1\times5}, \qquad recurrent\ layer: W_{5\times5}, b_{1\times5} \qquad output\ layer: V_{5\times1}, b_{1\times1}
$$

In [8]:
np.random.seed(0)

# Define our weights and biases
# Scale them down so values get through the tanh nonlinearity
i_weight = np.random.rand(1,5) / 5 - .1

h_weight = np.random.rand(5,5) / 5 - .1
h_bias = np.random.rand(1,5) / 5 - .1

# Tanh pushes values to between -1 and 1, so scale up the output weights
o_weight = np.random.rand(5,1) * 50
o_bias = np.random.rand(1,1)

### Demo

#### Inputs

In [9]:

# This will store the previous hidden state, since we'll need it to calculate the current hidden step
prev_hidden = None
sequence = data["tmax"].tail(3).to_numpy()

#### Outputs

In [10]:
# An array to store the output predictions
outputs = np.zeros(3)
# An array to store hidden states for use in backpropagation
hiddens = np.zeros((3, 5))


In [11]:

for i in range(3):
    # Get the input sequence at the given position
    x = sequence[i].reshape(1,1)

    # Multiply input by input weight
    xi = x @ i_weight
    if prev_hidden is not None:
        # Add previous hidden to input
        xh = xi + prev_hidden @ h_weight + h_bias
    else:
        xh = xi

    # Apply our activation function
    xh = np.tanh(xh)
    prev_hidden = xh
    hiddens[i,] = xh

    # Multiply by the output weight
    xo = xh @ o_weight + o_bias
    outputs[i] = xo

C:\Users\xiaophai\AppData\Local\Temp\ipykernel_28124\4045844613.py:20: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  outputs[i] = xo


In [12]:
outputs

array([74.31470595, 80.66149404, 77.67852446])

## Calculating loss

$$
loss = \frac{1}{2}\sum_i (actual_i - predict_i)^2
$$

$$
loss\_grad_i = \frac{\partial loss}{\partial predict_i} = (predict_i - actual_i)
$$

In [13]:
def mse(actual, predicted):
    return np.mean((actual-predicted)**2)

def mse_grad(actual, predicted):
    return (predicted - actual)

In [14]:
# Actual next day temperatures
actuals = np.array([70, 62, 65])

loss_grad = mse_grad(actuals, outputs)
loss_grad

array([ 4.31470595, 18.66149404, 12.67852446])

## Backward Pass

$$
\begin{split}
\frac{\partial E^t}{\partial W}
% 第一个等号
&=
\frac{\partial E^t}{\partial \boldsymbol{\eta}^t}
\frac{\partial \boldsymbol{\eta}^t}{\partial W}
\\
&=
\frac{\partial E^t}{\partial \boldsymbol{\eta}^t}
\frac{\partial W}{\partial W}\boldsymbol{s}^{t-1}
+
\frac{\partial E^t}{\partial \boldsymbol{\eta}^{t-1}}
\frac{\partial \boldsymbol{\eta}^{t-1}}{\partial W}
\\
&=
\frac{\partial E^t}{\partial \boldsymbol{\eta}^t}
\frac{\partial W}{\partial W}\boldsymbol{s}^{t-1}
+
\frac{\partial E^t}{\partial \boldsymbol{\eta}^{t-1}}
\frac{\partial W}{\partial W}\boldsymbol{s}^{t-2}
+
\frac{\partial E^t}{\partial \boldsymbol{\eta}^{t-2}}
\frac{\partial \boldsymbol{\eta}^{t-2}}{\partial W}
\\
&=
\frac{\partial E^t}{\partial \boldsymbol{\eta}^t}
\frac{\partial W}{\partial W}\boldsymbol{s}^{t-1}
+
\frac{\partial E^t}{\partial \boldsymbol{\eta}^{t-1}}
\frac{\partial W}{\partial W}\boldsymbol{s}^{t-2}
+
\frac{\partial E^t}{\partial \boldsymbol{\eta}^{t-2}}
\frac{\partial W}{\partial W}\boldsymbol{s}^{t-3}
+
\cdots
+
\frac{\partial E^t}{\partial \boldsymbol{\eta}^{2}}
\frac{\partial W}{\partial W}\boldsymbol{s}^{1}
+
\frac{\partial E^t}{\partial \boldsymbol{\eta}^{1}}
\frac{\partial W}{\partial W}\boldsymbol{s}^{0}
\end{split}
$$

$$
\begin{split}
\frac{\partial E^t}{\partial U}
% 第一个等号
&=
\frac{\partial E^t}{\partial \boldsymbol{\eta}^t}
\frac{\partial \boldsymbol{\eta}^t}{\partial U}
\\
&=
\frac{\partial E^t}{\partial \boldsymbol{\eta}^t}
\frac{\partial U\boldsymbol{x}^t}{\partial U}
+
\frac{\partial E^t}{\partial \boldsymbol{\eta}^{t-1}}
\frac{\partial \boldsymbol{\eta}^{t-1}}{\partial U}
\\
&=
\frac{\partial E^t}{\partial \boldsymbol{\eta}^t}
\frac{\partial U\boldsymbol{x}^t}{\partial U}
+
\frac{\partial E^t}{\partial \boldsymbol{\eta}^{t-1}}
\frac{\partial U\boldsymbol{x}^{t-1}}{\partial U}
+
\frac{\partial E^t}{\partial \boldsymbol{\eta}^{t-2}}
\frac{\partial \boldsymbol{\eta}^{t-2}}{\partial U}
\\
&=
\frac{\partial E^t}{\partial \boldsymbol{\eta}^t}
\frac{\partial U\boldsymbol{x}^t}{\partial U}
+
\frac{\partial E^t}{\partial \boldsymbol{\eta}^{t-1}}
\frac{\partial U\boldsymbol{x}^{t-1}}{\partial U}
+
\frac{\partial E^t}{\partial \boldsymbol{\eta}^{t-2}}
\frac{\partial U\boldsymbol{x}^{t-2}}{\partial U}
+
\cdots
+
\frac{\partial E^t}{\partial \boldsymbol{\eta}^{2}}
\frac{\partial U\boldsymbol{x}^{2}}{\partial U}
+
\frac{\partial E^t}{\partial \boldsymbol{\eta}^{1}}
\frac{\partial U\boldsymbol{x}^{1}}{\partial U}
\end{split}
$$

In [27]:
next_hidden = None

o_weight_grad, o_bias_grad, h_weight_grad, h_bias_grad, i_weight_grad = [0] * 5

for i in range(2, -1, -1):
    l_grad = loss_grad[i].reshape(1,1)

    o_weight_grad += hiddens[i][:,np.newaxis] @ l_grad
    o_bias_grad += np.mean(l_grad)

    o_grad = l_grad @ o_weight.T

    # Only add in the hidden gradient if a next sequence exists
    if next_hidden is not None:
        h_grad = o_grad + next_hidden @ h_weight.T
    else:
        h_grad = o_grad

    tanh_deriv = 1 - hiddens[i,:][np.newaxis,:] ** 2
    h_grad = np.multiply(h_grad, tanh_deriv)

    next_hidden = h_grad

    # Don't update the hidden weights for the first sequence position
    if i > 0:
        h_weight_grad += hiddens[i-1,:][:,np.newaxis] @ h_grad
        h_bias_grad += np.mean(h_grad)

    i_weight_grad += sequence[i].reshape(1,1).T @ h_grad

We can then use gradient descent to make parameter updates:

In [28]:
lr = 1e-6
# We'll divide the learning rate by the sequence length, since we were adding together the gradients
# This makes training the model more stable
lr = lr / 3

i_weight -= i_weight_grad * lr
h_weight -= h_weight_grad * lr
h_bias -= h_bias_grad * lr
o_weight -= o_weight_grad * lr
o_bias -= o_bias_grad * lr

## Full Implementation

We now know enough to do a full implementation of a complete network.  This code will mostly be the same as the forward and backward passes we already implemented.

We'll first load in and scale our data:

In [17]:
from sklearn.preprocessing import StandardScaler
import math

# Define predictors and target
PREDICTORS = ["tmax", "tmin", "rain"]
TARGET = "tmax_tomorrow"

# Scale our data to have mean 0
scaler = StandardScaler()
data[PREDICTORS] = scaler.fit_transform(data[PREDICTORS])

# Split into train, valid, test sets
np.random.seed(0)
split_data = np.split(data, [int(.7*len(data)), int(.85*len(data))])
(train_x, train_y), (valid_x, valid_y), (test_x, test_y) = [[d[PREDICTORS].to_numpy(), d[[TARGET]].to_numpy()] for d in split_data]

c:\Users\xiaophai\.conda\envs\python39\Lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


Then we can initialize our weights and biases.  We'll scale our parameters so they are relatively small.  This helps the network descend better:

In [18]:
def init_params(layer_conf):
    layers = []
    for i in range(1, len(layer_conf)):
        np.random.seed(0)
        k = 1/math.sqrt(layer_conf[i]["hidden"])
        i_weight = np.random.rand(layer_conf[i-1]["units"], layer_conf[i]["hidden"]) * 2 * k - k

        h_weight = np.random.rand(layer_conf[i]["hidden"], layer_conf[i]["hidden"]) * 2 * k - k
        h_bias = np.random.rand(1, layer_conf[i]["hidden"]) * 2 * k - k

        o_weight = np.random.rand(layer_conf[i]["hidden"], layer_conf[i]["output"]) * 2 * k - k
        o_bias = np.random.rand(1, layer_conf[i]["output"]) * 2 * k - k

        layers.append(
            [i_weight, h_weight, h_bias, o_weight, o_bias]
        )
    return layers

Then we'll write a forward pass:

In [19]:
def forward(x, layers):
    hiddens = []
    outputs = []
    for i in range(len(layers)):
        i_weight, h_weight, h_bias, o_weight, o_bias = layers[i]
        hidden = np.zeros((x.shape[0], i_weight.shape[1]))
        output = np.zeros((x.shape[0], o_weight.shape[1]))
        for j in range(x.shape[0]):
            input_x = x[j,:][np.newaxis,:] @ i_weight
            hidden_x = input_x + hidden[max(j-1,0),:][np.newaxis,:] @ h_weight + h_bias
            # Activation.  tanh avoids outputs getting larger and larger.
            hidden_x = np.tanh(hidden_x)
            # Store hidden for use in backprop
            hidden[j,:] = hidden_x

            # Output layer
            output_x = hidden_x @ o_weight + o_bias
            output[j,:] = output_x
        hiddens.append(hidden)
        outputs.append(output)
    return hiddens, outputs[-1]

And a backward pass:

In [20]:
def backward(layers, x, lr, grad, hiddens):
    for i in range(len(layers)):
        i_weight, h_weight, h_bias, o_weight, o_bias = layers[i]
        hidden = hiddens[i]
        next_h_grad = None
        i_weight_grad, h_weight_grad, h_bias_grad, o_weight_grad, o_bias_grad = [0] * 5

        for j in range(x.shape[0] - 1, -1, -1):
            # Add newaxis in the first dimension
            out_grad = grad[j,:][np.newaxis, :]

            # Output updates
            # np.newaxis creates a size 1 axis, in this case transposing matrix
            o_weight_grad += hidden[j,:][:, np.newaxis] @ out_grad
            o_bias_grad += out_grad

            # Propagate gradient to hidden unit
            h_grad = out_grad @ o_weight.T

            if j < x.shape[0] - 1:
                # Then we multiply the gradient by the hidden weights to pull gradient from next hidden state to current hidden state
                hh_grad = next_h_grad @ h_weight.T
                # Add the gradients together to combine output contribution and hidden contribution
                h_grad += hh_grad

            # Pull the gradient across the current hidden nonlinearity
            # derivative of tanh is 1 - tanh(x) ** 2
            # So we take the output of tanh (next hidden state), and plug in
            tanh_deriv = 1 - hidden[j][np.newaxis,:] ** 2

            # next_h_grad @ np.diag(tanh_deriv_next) multiplies each element of next_h_grad by the deriv
            # Effect is to pull value across nonlinearity
            h_grad = np.multiply(h_grad, tanh_deriv)

            # Store to compute h grad for previous sequence position
            next_h_grad = h_grad.copy()

            # If we're not at the very beginning
            if j > 0:
                # Multiply input from previous layer by post-nonlinearity grad at current layer
                h_weight_grad += hidden[j-1][:, np.newaxis] @ h_grad
                h_bias_grad += h_grad

            i_weight_grad += x[j,:][:,np.newaxis] @ h_grad

        # Normalize lr by number of sequence elements
        lr = lr / x.shape[0]
        i_weight -= i_weight_grad * lr
        h_weight -= h_weight_grad * lr
        h_bias -= h_bias_grad * lr
        o_weight -= o_weight_grad * lr
        o_bias -= o_bias_grad * lr
        layers[i] = [i_weight, h_weight, h_bias, o_weight, o_bias]
    return layers

We can end by setting up a training loop and measuring error:

In [21]:
epochs = 101
lr = 1e-5

layer_conf = [
    {"type":"input", "units": 3},
    {"type": "rnn", "hidden": 4, "output": 1}
]
layers = init_params(layer_conf)

for epoch in range(epochs):
    sequence_len = 7
    epoch_loss = 0
    for j in range(train_x.shape[0] - sequence_len):
        seq_x = train_x[j:(j+sequence_len),]
        seq_y = train_y[j:(j+sequence_len),]
        hiddens, outputs = forward(seq_x, layers)
        grad = mse_grad(seq_y, outputs)
        params = backward(layers, seq_x, lr, grad, hiddens)
        epoch_loss += mse(seq_y, outputs)

    if epoch % 10 == 0:
        sequence_len = 7
        valid_loss = 0
        for j in range(valid_x.shape[0] - sequence_len):
            seq_x = valid_x[j:(j+sequence_len),]
            seq_y = valid_y[j:(j+sequence_len),]
            _, outputs = forward(seq_x, layers)
            valid_loss += mse(seq_y, outputs)

        print(f"Epoch: {epoch} train loss {epoch_loss / len(train_x)} valid loss {valid_loss / len(valid_x)}")

Epoch: 0 train loss 3122.5944001445105 valid loss 2171.3186862102048
Epoch: 10 train loss 84.78519304952819 valid loss 81.34087884698089
Epoch: 20 train loss 52.841666692401546 valid loss 52.309411675624744
Epoch: 30 train loss 34.89142680576197 valid loss 34.42998362512595
Epoch: 40 train loss 33.90674797419208 valid loss 33.78966022195152
Epoch: 50 train loss 30.593193275313336 valid loss 30.568271740103118
Epoch: 60 train loss 27.767753755456212 valid loss 27.466770094462962
Epoch: 70 train loss 27.987707979297117 valid loss 27.581650000895717
Epoch: 80 train loss 28.42712118570578 valid loss 27.791959394169872
Epoch: 90 train loss 26.598198926416888 valid loss 25.80674232379205
Epoch: 100 train loss 25.263986813543617 valid loss 24.43551751035542
